In [17]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],
  api_version = "2023-03-15-preview"
)
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

In [26]:
messages = [
    {"role": "user", "content": "Encontre um bom curso para um estudante iniciante aprender Azure."}
]

In [27]:
functions = [
   {
      "name": "search_courses",
      "description": "Recupera cursos do índice de busca com base nos parâmetros fornecidos",
      "parameters": {
         "type": "object",
         "properties": {
            "role": {
               "type": "string",
               "description": "O papel do aprendiz (por exemplo, desenvolvedor, cientista de dados, estudante, etc.)"
            },
            "product": {
               "type": "string",
               "description": "O produto que a lição está cobrindo (por exemplo, Azure, Power BI, etc.)"
            },
            "level": {
               "type": "string",
               "description": "O nível de experiência que o aprendiz tem antes de fazer o curso (por exemplo, iniciante, intermediário, avançado)"
            }
         },
         "required": ["role"]
      }
   }
]

In [29]:
response = client.chat.completions.create(
    model=model, 
    messages=messages,
    functions=functions, 
    function_call="auto"
) 

response_message = response.choices[0].message

In [30]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level,
        "locale": "pt-br"
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)

In [31]:
if response_message.function_call.name:
    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    translate = [
        {"role": "user", "content": "Traduza para o ingles: " + response_message.function_call.arguments }
    ]
    response_translate = client.chat.completions.create(model=model, messages=translate) 

    function_args = json.loads(response_translate.choices[0].message.content)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )

Output of function call:
[{'title': 'Descrever conceitos de criptografia', 'url': 'https://learn.microsoft.com/pt-br/training/modules/describe-concepts-of-cryptography/?WT.mc_id=api_CatalogApi'}, {'title': 'Introdução à classificação de áudio com o TensorFlow', 'url': 'https://learn.microsoft.com/pt-br/training/modules/intro-audio-classification-tensorflow/?WT.mc_id=api_CatalogApi'}, {'title': 'Criar um modelo de dados de alto desempenho no banco de dados SQL do Azure com o Azure Data Studio', 'url': 'https://learn.microsoft.com/pt-br/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi'}, {'title': 'Princípios básicos da Fala de IA do Azure', 'url': 'https://learn.microsoft.com/pt-br/training/modules/recognize-synthesize-speech/?WT.mc_id=api_CatalogApi'}, {'title': 'Introdução ao Microsoft Cloud Adoption Framework para o Azure', 'url': 'https://learn.microsoft.com/pt-br/training/modules/cloud-adoption-framework-getting-started/?WT.mc_id=api_CatalogApi'}]
<class 'str'

In [32]:
second_response = client.chat.completions.create(
    messages=messages,
    model=model,
    function_call="auto",
    functions=functions,
    temperature=0
)  

print(second_response.choices[0].message)

ChatCompletionMessage(content='Aqui estão alguns cursos recomendados para um estudante iniciante aprender Azure:\n\n1. **[Descrever conceitos de criptografia](https://learn.microsoft.com/pt-br/training/modules/describe-concepts-of-cryptography/?WT.mc_id=api_CatalogApi)**\n   \n2. **[Introdução à classificação de áudio com o TensorFlow](https://learn.microsoft.com/pt-br/training/modules/intro-audio-classification-tensorflow/?WT.mc_id=api_CatalogApi)**\n\n3. **[Criar um modelo de dados de alto desempenho no banco de dados SQL do Azure com o Azure Data Studio](https://learn.microsoft.com/pt-br/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi)**\n\n4. **[Princípios básicos da Fala de IA do Azure](https://learn.microsoft.com/pt-br/training/modules/recognize-synthesize-speech/?WT.mc_id=api_CatalogApi)**\n\n5. **[Introdução ao Microsoft Cloud Adoption Framework para o Azure](https://learn.microsoft.com/pt-br/training/modules/cloud-adoption-framework-getting-started/?WT.m